In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkSession").getOrCreate()

In [10]:
from graphframes import *
import hashlib

ModuleNotFoundError: No module named 'graphframes'

In [6]:
df = spark.read.load("test2/",
                     format="csv", sep="\t", inferSchema="true", header="false")

In [7]:
df.show(10)

+--------------------+--------------------+
|                 _c0|                 _c1|
+--------------------+--------------------+
|           002224.cn|       hd.hstong.com|
|           002366.cn|       hd.hstong.com|
|           002366.cn|       hd.hstong.com|
|01278.outlookrefl...|           nginx.org|
|            0311.top|cloudcdn.dopa.com.cn|
|            0311.top|cloudcdn.dopa.com.cn|
|            0311.top|ms.ok365.com?dm=0...|
|            0311.top|cloudcdn.dopa.com.cn|
|            0311.top|cloudcdn.dopa.com.cn|
|            0311.top|cloudcdn.dopa.com.cn|
+--------------------+--------------------+
only showing top 10 rows



In [8]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [9]:
df.count()

320207